In [9]:
############################# IMPORT STATEMENTS ########################################################
#Import Python modules
import numpy as np
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import cv2
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from keras.callbacks import ModelCheckpoint, EarlyStopping

#For local CPU usage:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

#Import Keras modules
from keras.layers import Dense, Flatten, Input, Dropout, Conv1D, Conv2D, LSTM, Concatenate, Reshape, MaxPool1D, MaxPool2D, BatchNormalization
from keras import Model, Sequential
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
import keras.backend as K
import tensorflow as tf
from scipy import stats

In [10]:
############################# SETUP PROJECT PARAMETERS ########################################################
LOAD_PROGRESS_FROM_MODEL = False
SAVE_PROGRESS_TO_MODEL = True

IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

PATH_TO_FOLDER = 'C:/Users/Tobias/Desktop/Master-Thesis/Data/AFEW/002'
PATH_TO_DATA = 'C:/Users/Tobias/Desktop/Master-Thesis/Data/AFEW'
FOLDER = "/002"
DATA_DIR_PREDICT = 'C:/Users/Tobias/Desktop/Master-Thesis/Data/facesdb/s005/tif/training/'
DISPLAY_IMG_NAME = '00000.png'
IMG_FORMAT = '.png'

batch_size = 32
num_folds = 3   # needs to be at least 2 for train and test

CROSS_VALIDATION = False
EPOCHS = 20
EPOCHS_CROSS = 10

# Approach:

### 1. Face detection
using MTCNN (Simultaneous face detection, face alignment, bounding boxing and landmark detection)

### 2. Highlighting faces
draw the bounding box in an image and plot it - to check out the result

### 3. Face extraction
extracting the face according to the identified bounding box

### 4. Face recognition
Using the VGGFace pretrained Resnet50 model to recognize emotions (training + prediction)

## 0. Get data

In [11]:
# for now reading the RGB values of the image (maybe in future just gray-scale)
def get_image_from_filename(path_to_folder, filename):
    image = cv2.cvtColor(cv2.imread(os.path.join(path_to_folder, filename)), cv2.COLOR_BGR2RGB)
    return image

def get_all_images(path_to_data):
    images = []
    for folder in os.listdir(path_to_data):
        for filename in os.listdir(os.path.join(path_to_data, folder)):
            img = cv2.imread(os.path.join(path_to_data, folder, filename))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)
    return images

In [12]:
# sructure data according to emotions (necessary to read in labels)
def get_labels_from_folder(path_to_data):
    labels = []
    
    for folder in os.listdir(path_to_data):
        with open(os.path.join(path_to_data, folder, folder + ".json")) as p:
            data = json.load(p)
        frames = data['frames']
    
        for key, value in frames.items():
            labels.append([value['valence'], value['arousal']])

    return labels

In [13]:
# split data into training, cv and test
def get_splitted_data(path_to_data):
    labels = get_labels_from_folder(path_to_data)
    filenames = get_all_images(path_to_data)
    
    filenames_shuffled_np, labels_shuffled_np = shuffle(filenames, labels)
    X_training, X_test, Y_training, Y_test = train_test_split(filenames_shuffled_np, labels_shuffled_np, test_size=0.2, random_state=1)
    
    return X_training, X_test, Y_training, Y_test

 
## 1. Face detection

In [6]:
detector = MTCNN()

def detect_face(image):
    face = detector.detect_faces(image)
    return face

def detect_faces(images):
    faces = []  
    for img in images:
        face = detector.detect_faces(img)
        if len(face) == 1:
            faces.append(face)  ## just use the face with the highest detection probability
        elif len(face) > 1:
            faces.append(face[0])
        else:
            faces.append([]) ### no face was detected
    return faces

In [7]:
image = get_image_from_filename(PATH_TO_FOLDER, DISPLAY_IMG_NAME)
face = detect_face(image)
print(face)
len(face)

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


## 2. Higlighting face

In [ ]:
def highlight_face(path_to_folder, filename):
  # display image
    image = get_image_from_filename(path_to_folder, filename)
    plt.imshow(image)

    ax = plt.gca()
    face = detect_face(image)
    
    # for each face in the image, draw a rectangle based on coordinates
    for elem in face:
        x, y, width, height = elem['box']
        face_border = Rectangle((x, y), width, height,
                          fill=False, color='red')
        ax.add_patch(face_border)
    plt.show()

In [ ]:
highlight_face(PATH_TO_FOLDER, DISPLAY_IMG_NAME)

## 3. Face extraction

In [ ]:
def extract_face_from_image(image, required_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    face = detect_face(image) # content of face is a python dict

    # extract the bounding box from the requested face
    box = np.asarray(face[0]['box'])
    box[box < 0] = 0
    x1, y1, width, height =  box
    
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face_boundary = image[y1:y2, x1:x2]

    # resize pixels to the model size
    face_image = Image.fromarray(face_boundary)
    face_image = face_image.resize(required_size)
    face_array = asarray(face_image)
        
    return face_array


def extract_face_from_images(images, required_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    faces = detect_faces(images)
    face_images = []
    
    for i in range(len(images)):
        if faces[i-1] == []: # No face detected
            face_image = Image.fromarray(images[i-1])
            face_image = face_image.resize(required_size)
            face_array = asarray(face_image)
            face_images.append(face_array)
        else:
            # extract the bounding box from the requested face
            if type(faces[i-1]) is list:  # checks whether more than one face was detected
                box = np.asarray(faces[i-1][0]['box'])
                box[box < 0] = 0
                x1, y1, width, height = box
            else:
                box = np.asarray(faces[i-1]['box'])
                box[box < 0] = 0
                x1, y1, width, height = box

            x2, y2 = x1 + width, y1 + height
            # extract the face
            face_boundary = images[i-1][y1:y2, x1:x2]

            # resize pixels to the model size
            face_image = Image.fromarray(face_boundary)
            face_image = face_image.resize(required_size)
            face_array = asarray(face_image)
            face_images.append(face_array)
            
    return face_images

In [8]:
# Display the first face from the extracted faces
image = get_image_from_filename(PATH_TO_FOLDER, DISPLAY_IMG_NAME)
extracted_face = extract_face_from_image(image)
plt.imshow(extracted_face)
plt.show()
extracted_face.shape

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


## 4. Face recognition -> Emotions

In [12]:
def get_face_embedding(faces):
    samples = asarray(faces, 'float32')

    # prepare the data for the model
    samples = preprocess_input(samples, version=2)

    # create a vggface model object
    model = VGGFace(model='resnet50',
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='avg')
    
    # perform prediction
    return model.predict(samples)

In [13]:
X_training, X_test, Y_train, Y_test = get_splitted_data(PATH_TO_DATA)

In [14]:
print(len(X_training))
print(len(X_test))
print(len(Y_train))
print(len(Y_test))

80
20
80
20


In [ ]:
X_train_faces = extract_face_from_images(X_training)
X_test_faces = extract_face_from_images(X_test)

X_train_embeddings = get_face_embedding(X_train_faces)
X_test_embeddings = get_face_embedding(X_test_faces)

In [ ]:
np.savetxt('X_train_embeddings.txt', X_train_embeddings)
np.savetxt('X_test_embeddings.txt', X_test_embeddings)

np.savetxt('Y_train.txt', Y_train)
np.savetxt('Y_test.txt', Y_test)

In [ ]:
X_train_embeddings = np.loadtxt('X_train_embeddings.txt')
X_test_embeddings = np.loadtxt('X_test_embeddings.txt')

Y_train = np.loadtxt('Y_train.txt')
Y_test = np.loadtxt('Y_test.txt')

In [ ]:
#the model returns a vector, which represents the features of a face
print(X_train_embeddings)
print(X_train_embeddings.shape)
Y_train = np.asarray(Y_train)

In [ ]:
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def corr(y_true, y_pred):
    #normalise
    n_y_true = (y_true - K.mean(y_true[:])) / K.std(y_true[:])
    n_y_pred = (y_pred - K.mean(y_pred[:])) / K.std(y_pred[:])  

    top=K.sum((n_y_true[:]-K.mean(n_y_true[:]))*(n_y_pred[:]-K.mean(n_y_pred[:])),axis=[-1,-2])
    bottom=K.sqrt(K.sum(K.pow((n_y_true[:]-K.mean(n_y_true[:])),2),axis=[-1,-2])*K.sum(K.pow(n_y_pred[:]-K.mean(n_y_pred[:]),2),axis=[-1,-2]))

    result=top/bottom
    return K.mean(result)

In [ ]:
def corr_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)



In [ ]:
def model_top(input_shape):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_dim = input_shape))
    model.add(Dropout(0.1))

    model.add(Dense(256, activation='relu'))

    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim = 2, activation='tanh')) 

    return model

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

inputs = np.concatenate((X_train_embeddings, X_test_embeddings), axis=0)
targets = np.append(Y_train, Y_test, axis = 0)

In [ ]:
inputs.shape

In [ ]:
mc_best = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10) # waiting for 10 consecutive epochs that don't reduce the val_loss

In [ ]:
# K-fold Cross Validation model evaluation
history_accuracy = []
history_val_accuracy = []
history_loss = []
history_val_loss = []
history_corr = []
history_val_corr = []
history_rmse = []
history_val_rmse = []


if CROSS_VALIDATION == True:
    for train, test in kfold.split(inputs, targets):
        model = model_top(2048)

        if LOAD_PROGRESS_FROM_MODEL:
            model.load_weights("model_checkpoints/model_top.h5")
            print("Loaded model from disk")

        model.summary()
        model.compile(loss = corr_loss, optimizer = "adam", metrics = ["accuracy", rmse, corr])

        # train model  
        scores = model.fit(inputs[train], targets[train], steps_per_epoch = int(len(train) // batch_size), 
                           epochs=EPOCHS_CROSS, verbose=1, validation_data = (inputs[test], targets[test]),
                           validation_steps = int(len(test) // batch_size), callbacks = [mc_best, mc_es])

        history_accuracy.extend(scores.history['accuracy'])
        history_val_accuracy.extend(scores.history['val_accuracy'])
        history_loss.extend(scores.history['loss'])
        history_val_loss.extend(scores.history['val_loss'])
        history_corr.extend(scores.history['corr'])
        history_val_corr.extend(scores.history['val_corr'])
        history_rmse.extend(scores.history['rmse'])
        history_val_rmse.extend(scores.history['val_rmse'])
else:
    model = model_top(2048)

    if LOAD_PROGRESS_FROM_MODEL:
        model.load_weights("model_checkpoints/model_top.h5")
        print("Loaded model from disk")

    model.summary()
    opt = SGD(lr=0.01)  # optimizer = "adam"
    model.compile(loss = corr_loss, optimizer = opt, metrics = ["accuracy", rmse, corr])

    # train model  
    scores = model.fit(X_train_embeddings, Y_train, epochs=EPOCHS, verbose=1, 
                       validation_data = (X_test_embeddings, Y_test), callbacks = [mc_best, mc_es])
        # steps_per_epoch = int(len(X_train_embeddings) // batch_size)
        # validation_steps = int(len(X_test_embeddings) // batch_size)

    history_accuracy.extend(scores.history['accuracy'])
    history_val_accuracy.extend(scores.history['val_accuracy'])
    history_loss.extend(scores.history['loss'])
    history_val_loss.extend(scores.history['val_loss'])
    history_corr.extend(scores.history['corr'])
    history_val_corr.extend(scores.history['val_corr'])
    history_rmse.extend(scores.history['rmse'])
    history_val_rmse.extend(scores.history['val_rmse'])

In [ ]:
if SAVE_PROGRESS_TO_MODEL:
    model.save_weights("model_checkpoints/model_top.h5")
    print("Saved model to disk")

In [ ]:
# summarize history for accuracy
plt.figure(1)
plt.plot(history_accuracy)
plt.plot(history_val_accuracy)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model_checkpoints/accuracy.png')
plt.show()

# summarize history for CORR
plt.figure(2)
plt.plot(history_corr)
plt.plot(history_val_corr)
plt.title('model correlation(CORR)')
plt.ylabel('correlation')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model_checkpoints/correlation.png')
plt.show()

# summarize history for loss
plt.figure(3)
plt.plot(history_loss)
plt.plot(history_val_loss)
plt.title('model correlation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model_checkpoints/correlation_loss.png')
plt.show()

# summarize history for RMSE
plt.figure(4)
plt.plot(history_rmse)
plt.plot(history_val_rmse)
plt.title('model root_mean_squared_error')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model_checkpoints/rmse.png')
plt.show()

In [136]:
### PREDICTION

In [138]:
emotion_scores = model.predict(X_test_embeddings)

In [139]:
# the model returns the emtions in 2-dimensional Valance-Arousal space
print(emotion_scores)

[[0.99998695 0.99983275]
 [0.9999988  0.9999997 ]
 [0.99997723 0.99977726]
 [0.99999726 0.9999995 ]
 [0.99999803 0.9999695 ]
 [0.99999785 0.9999996 ]
 [0.999983   0.9998684 ]
 [0.9999985  0.9999771 ]
 [0.999995   0.99993503]
 [0.9999882  0.99996454]
 [0.9999948  0.99986416]
 [0.99999696 0.99999934]
 [0.9999894  0.99999905]
 [0.99999064 0.9998894 ]
 [0.99999785 0.99999964]
 [0.99999636 0.99984926]
 [0.9999278  0.9996679 ]
 [0.9999962  0.9998706 ]
 [0.99999744 0.9999994 ]
 [0.99998343 0.9999043 ]]
